In [248]:
import unittest

In [249]:
def load_data(n = None):
    # Default dataset
    males = {'0': ['7', '5', '6', '4'],
        '1': ['5', '4', '6', '7'],
        '2': ['4', '5', '6', '7'],
        '3': ['4', '5', '6', '7']}

    females = {'4': ['0', '1', '2', '3'],
          '5': ['0', '1', '2', '3'],
          '6': ['0', '1', '2', '3'],
          '7': ['0', '1', '2', '3']}

    # Dataset 1
    if n == 0:
        males = {'k': ['b', 'c', 'a'],
                'l': ['a', 'c', 'b'],
                'm': ['a', 'b', 'c']}

        females = {'a': ['k', 'l', 'm'],
                  'b': ['l', 'm', 'k'],
                  'c': ['m', 'l', 'k']}            
    return males, females

In [300]:
def is_empty(n):
    return len(n) == 0

class StableMarriageTestCase(unittest.TestCase):
    males_1 = {'0': ['7', '5', '6', '4'],
        '1': ['5', '4', '6', '7'],
        '2': ['4', '5', '6', '7'],
        '3': ['4', '5', '6', '7']}

    females_1 = {'4': ['0', '1', '2', '3'],
          '5': ['0', '1', '2', '3'],
          '6': ['0', '1', '2', '3'],
          '7': ['0', '1', '2', '3']}

    males_2 = {'k': ['b', 'c', 'a'],
            'l': ['a', 'c', 'b'],
            'm': ['a', 'b', 'c']}

    females_2 = {'a': ['k', 'l', 'm'],
              'b': ['l', 'm', 'k'],
              'c': ['m', 'l', 'k']}   
    
    def test_load_default_data(self):
        males, females = load_data()
        self.assertEqual(males, self.males_1)
        self.assertEqual(females, self.females_1)

    def test_load_data(self):
        males, females = load_data(0)
        self.assertEqual(males, self.males_2)
        self.assertEqual(females, self.females_2)

if __name__ == '__main__':
    unittest.main(argv = ['ignore-first-argv'], exit = False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [251]:
# males = {'k': ['b', 'c', 'a'],
#         'l': ['a', 'c', 'b'],
#         'm': ['a', 'b', 'c']}

# females = {'a': ['k', 'l', 'm'],
#           'b': ['l', 'm', 'k'],
#           'c': ['m', 'l', 'k']}
# # Expected outcome: {'b': 'k', 'a': 'l', 'c': 'k'}

In [252]:
males = {'0': ['7', '5', '6', '4'],
        '1': ['5', '4', '6', '7'],
        '2': ['4', '5', '6', '7'],
        '3': ['4', '5', '6', '7']}

females = {'4': ['0', '1', '2', '3'],
          '5': ['0', '1', '2', '3'],
          '6': ['0', '1', '2', '3'],
          '7': ['0', '1', '2', '3']}
# {'7': '0', '5': '1', '4': '2', '6': '3'}

In [253]:
output = {}

epochs = len(males.keys())

proposer = {}


def match(i):
    for male in males:
        if male in proposer:
            if proposer[male]:
                continue
        choice = males[male][i]
        print('guy {} pick girl {}'.format(male, choice))
        if choice in output:
            print('girl {} is choosen'.format(choice))
            picked_male = output[choice]
            if females[choice].index(picked_male) < females[choice].index(male):
                # Smaller index means higher preference
                print('girl {} picked {} instead of {}'.format(choice, picked_male, male))
                output[choice] = picked_male
                proposer[male] = False
                proposer[picked_male] = True
            else:
                proposer[male] = True
                proposer[picked_male] = False
        else:
            output[choice] = male
            proposer[male] = True
    print(output, proposer)

while not all(proposer.values()) or len(proposer.values()) != epochs:
    for i in range(epochs):
        match(i)

guy 0 pick girl 7
guy 1 pick girl 5
guy 2 pick girl 4
guy 3 pick girl 4
girl 4 is choosen
girl 4 picked 2 instead of 3
{'7': '0', '5': '1', '4': '2'} {'0': True, '1': True, '2': True, '3': False}
guy 3 pick girl 5
girl 5 is choosen
girl 5 picked 1 instead of 3
{'7': '0', '5': '1', '4': '2'} {'0': True, '1': True, '2': True, '3': False}
guy 3 pick girl 6
{'7': '0', '5': '1', '4': '2', '6': '3'} {'0': True, '1': True, '2': True, '3': True}
{'7': '0', '5': '1', '4': '2', '6': '3'} {'0': True, '1': True, '2': True, '3': True}


In [254]:
class Male:
    def __init__(self, name, preference):
        self.name = name
        self.preference = preference
        self.is_matched = False
        self.partner = None
        self.index = 0 # Index to hold the position of the proposed wives
    
    def propose(self):
        female = self.preference[self.index]
        self.index += 1
        return female
    
    def is_available(self):
        return not self.is_matched

    def __str__(self):
        if self.is_matched:
            return '{} is matched to {}'.format(self.name, self.partner.name)
        return '{} is single'.format(self.name)

In [263]:
class Female:
    def __init__(self, name, preference):
        self.name = name
        self.preference = preference
        self.is_matched = False
        self.partner = None
    
    def is_available(self):
        return not self.is_matched

    def rank(self, m1, m2):
        m1_score = self.preference.index(m1)
        m2_score = self.preference.index(m2)

        # Lower index means higher preference
        return m1 if m1_score < m2_score else m2

    def __str__(self):
        if self.is_matched:
            return '{} is matched to {}'.format(self.name, self.partner.name)
        return '{} is single'.format(self.name)

In [297]:
class Matches:
    def __init__(self, males, females):
        self.matches = {}

        self.choices = len(males)
        self.males = list(males.keys())
        self.females = list(females.keys())
        self.match_count = 0
        
        for i in males:
            self.matches[i] = Male(i, males[i])

        for j in females:
            self.matches[j] = Female(j, females[j])
        
    def engage(self, male_name, female_name):
        male = self.matches[male_name]
        female = self.matches[female_name]
        
        male.is_matched = True
        male.partner = female
        
        female.is_matched = True
        female.partner = male
        
        self.match_count += 1
    
    def breakup(self, male_name, female_name):
        male = self.matches[male_name]
        female = self.matches[female_name]
        
        male.is_matched = False
        male.partner = None
        
        female.is_matched = False
        female.partner = None

        self.match_count -= 1
    
    def match(self):
        for m in self.males:

            male = self.matches[m]
            if not male.is_available():
                continue
            female = self.matches[male.propose()]
            
            if female.is_available():
                self.engage(male.name, female.name)
            else:
                new_male_name = female.rank(male.name, female.partner.name)
                self.breakup(female.partner.name, new_male_name)
                self.engage(new_male_name, female.name)
        if self.match_count < self.choices:
            self.match()
        return self.matches
    
    def sets(self):
        matches = {}
        for i in self.matches:
            match = self.matches[i]
            print(match.name, match.partner.name)
            matches[frozenset([match.name, match.partner.name])] = True
        return list(matches.keys())

In [298]:
male_data, female_data = load_data()
male_data, female_data
matches = Matches(male_data, female_data)
out = matches.match()
for i in out:
    print(out[i])

matches.sets()

0 is matched to 7
1 is matched to 5
2 is matched to 4
3 is matched to 6
4 is matched to 2
5 is matched to 1
6 is matched to 3
7 is matched to 0
0 7
1 5
2 4
3 6
4 2
5 1
6 3
7 0


[frozenset({'0', '7'}),
 frozenset({'1', '5'}),
 frozenset({'2', '4'}),
 frozenset({'3', '6'})]